### Training and testing of the DistilBert model on the train dataset (Reddit book reviews)

In [2]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
import numpy as np
import evaluate


In [3]:
train_dataset = load_dataset("csv", data_files="books_train.csv")["train"]
test_dataset = load_dataset("csv", data_files="books_test.csv")["train"]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
   return tokenizer(examples["body"], truncation=True)
 
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)


In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)
   f1 = load_f1.compute(predictions=predictions, references=labels)
   return {"accuracy": accuracy, "f1": f1}

In [9]:
notebook_login()

In [10]:
repo_name = "finetuning-distibert-Reddit-books"
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


/tmp/ipykernel_6488/1078629372.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=26, training_loss=0.6231604356032151, metrics={'train_runtime': 21.1222, 'train_samples_per_second': 18.937, 'train_steps_per_second': 1.231, 'total_flos': 51536027486496.0, 'train_loss': 0.6231604356032151, 'epoch': 2.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.5824851393699646,
 'eval_accuracy': {'accuracy': 0.72},
 'eval_f1': {'f1': 0.8372093023255814},
 'eval_runtime': 3.4556,
 'eval_samples_per_second': 28.939,
 'eval_steps_per_second': 2.026,
 'epoch': 2.0}

In [16]:
trainer.evaluate()

{'eval_loss': 0.584126353263855,
 'eval_accuracy': {'accuracy': 0.72},
 'eval_f1': {'f1': 0.8372093023255814},
 'eval_runtime': 3.1473,
 'eval_samples_per_second': 31.774,
 'eval_steps_per_second': 2.224,
 'epoch': 2.0}

## The accuracy is 0.72, while the F1-score is 0.8372093023255814